In [ ]:
import numpy as np
import pandas as pd
import pyspark as ps
import matplotlib.pyplot as plt
import csv

import unicodedata

from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from gensim import corpora, models

import pyLDAvis.gensim

import pickle

In [ ]:
df = pickle.load( open( "processed_tweets.pkl", "rb" ))

In [ ]:
from collections import Counter

def compare(s, t):
    return Counter(s) == Counter(t)

def match_dict(tweet, dict_):
    """returns keywords that match in string
    tweet: tweet to find keywords in
    dict_: list of keywords
    """
    doc = [sublist for sublist in dict_ if compare(list(filter(lambda x: x in tweet, sublist)), sublist) ]
    return doc#at least one match

In [ ]:
df['keywords'] = df[df.lang == 'en']['tokenized'].map(lambda x: match_dict(x, en_dict))

In [ ]:
english_filtered[english_filtered.map(lambda x : len(x) > 0)]

In [ ]:
#move this above
import re
s = "string. With. Punctuation? ! àéè. : ; "
s = re.sub(r'[^\w\s]','',s)
s

## LDA starts here

In [ ]:
dictionary = corpora.Dictionary(english.tokenized)

dictionary.filter_extremes(no_below=3, no_above=.5)

texts = english.tokenized.tolist()

corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
ldamodel = models.LdaModel.load('models/general_lda_ennegative_neutral.model') #retrive lda model

In [ ]:
ldamodel.show_topics(num_topics=20)

In [ ]:
vis_data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

In [ ]:
pyLDAvis.display(vis_data)

In [ ]:
sent_to_cluster = list()
for n,doc in enumerate(corpus):
    if doc:
        cluster = max(ldamodel[doc],key=lambda x:x[1])
        sent_to_cluster.append(cluster[0])

## Same for french etc:

In [ ]:
french_filtered_2 = french_filtered.tokenized.map(lambda x: [re.sub(r'[^\w\s]','',s) for s in x])

In [ ]:
french_filtered_2.head()

In [ ]:
fr_dictionary = corpora.Dictionary(french_filtered_2)
fr_dictionary.filter_extremes(no_below=5, no_above=.3)
fr_texts = french_filtered_2.tolist()
fr_corpus = [fr_dictionary.doc2bow(text) for text in fr_texts]

ldamodel_2 = models.LdaMulticore(fr_corpus, id2word=fr_dictionary, num_topics=10, workers=3, iterations=100, passes=5)

vis_data = pyLDAvis.gensim.prepare(ldamodel_2, fr_corpus, fr_dictionary)

pyLDAvis.display(vis_data)

## Same for german:

In [ ]:
german_filtered_2 = german_filtered.tokenized.map(lambda x: [re.sub(r'[^\w\s]','',s) for s in x])

In [ ]:
ger_dictionary = corpora.Dictionary(german_filtered_2)
ger_dictionary.filter_extremes(no_below=3, no_above=.5)
ger_texts = german_filtered_2.tolist()
ger_corpus = [ger_dictionary.doc2bow(text) for text in ger_texts]

ldamodel_2 = models.LdaMulticore(ger_corpus, id2word=ger_dictionary, num_topics=5, workers=3, iterations=100, passes=5)

vis_data = pyLDAvis.gensim.prepare(ldamodel_2, ger_corpus, ger_dictionary)

pyLDAvis.display(vis_data)